### Querying Tabular Data
Lots of data and information is stored in tabular data, whether it be csvs, excel sheets, or SQL tables. We already covered talking to SQL, so let's focus on the other two.

Options available for querying tabular data:
- Document Loading - Load the data into a Document and then index it as you would other text/unstructured data. For this, you should use a document loader like the CSVLoader and then you should create an Index over that data, and query it that way.
- Querying - If you have more numeric tabular data, or have a large amount of data and don't want to index it, you can also use a language model to interact with it directly.
- Chains - If you are just getting started, and you have relatively small/simple tabular data, you should get started with chains. Chains are a sequence of predetermined steps, so they are good to get started with as they give you more control and let you understand what is happening better.
- Agents - Agents are more complex, and involve multiple queries to the LLM to understand what to do. The downside of agents are that you have less control. The upside is that they are more powerful, which allows you to use them on larger databases and more complex schemas.

Examples of chains and agents are covered in AskSQL notebook, where we used SQLAgents and SQLDatabaseChain.  This notebook covers the concept on CSV Agent



#### Set Environment Variables

In [1]:
import os  
import json  
import openai
from Utilities.envVars import *

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"https://{OpenAiService}.openai.azure.com"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci

In [2]:
# Import the needed Python packages
import urllib
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from IPython.display import display, HTML
from langchain.agents import create_csv_agent

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
        baseUrl = f"https://{OpenAiService}.openai.azure.com"
        openai.api_type = "azure"
        openai.api_key = OpenAiKey
        openai.api_version = OpenAiVersion
        openai.api_base = f"https://{OpenAiService}.openai.azure.com"

        llm = AzureChatOpenAI(
                    openai_api_base=baseUrl,
                    openai_api_version=OpenAiVersion,
                    deployment_name=OpenAiChat,
                    temperature=temperature,
                    openai_api_key=OpenAiKey,
                    openai_api_type="azure",
                    max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(model=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
        
elif embeddingModelType == "openai":
        openai.api_type = "open_ai"
        openai.api_base = "https://api.openai.com/v1"
        openai.api_version = '2020-11-07' 
        openai.api_key = OpenAiApiKey
        llm = ChatOpenAI(temperature=temperature,
                openai_api_key=OpenAiApiKey,
                max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)


This shows how to use agents to interact with a csv. It is mostly optimized for question answering.
NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful.


In [7]:
# Create CSV Agent and reference the Covid19 Dataset
csvAgent = create_csv_agent(llm, './Data/CSV/Covid19History.csv', verbose=False)

In [8]:
csvAgent.run("how many rows are there?")

'There are 20780 rows in the dataframe.'

In [10]:
answer = csvAgent.run("How may patients were hospitalized during July 2020 in Texas?")
print(answer)

31
0.0


Knowing that langchain csv_agent is using Pandas Dataframe under the hood, what if we create pandas dataframe agent to test the same query?

In [11]:
from langchain.agents import create_pandas_dataframe_agent
import pandas as pd

csvPromptPrefix = """
First set the pandas display options to show all the columns, get the column names, then answer the question.
Use pandasql and SQL queries to interact with the dataframe.

"""

csvPromptSuffx = """
- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result. 
- If you still cannot arrive to a consistent result, say that you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n". In the explanation, mention the column names that you used to get to the final answer. 
"""

question = "How may patients were hospitalized during July 2020 nationwide as the total of all states?"
df = pd.read_csv("./Data/CSV/Covid19History.csv").fillna(value = 0)
pandasAgent = create_pandas_dataframe_agent(llm=llm,df=df,verbose=False)
#answer = pandasAgent.run(csvPromptPrefix + question + csvPromptSuffx)
answer = pandasAgent.run(question)
print(answer)

5,535,849.0
